In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df = pd.read_csv('../data/current.csv')
df = df.dropna()
df['avgPowerplayToi'] = df['avgPowerplayToi'].apply(lambda x: datetime.strptime(str(x), "%M:%S").second + datetime.strptime(str(x), "%M:%S").minute * 60)
df['position'].replace(['C', 'R', 'L', 'D'], [3, 2, 2, 1], inplace=True)
df.head()

,skaterFullName,ppPoints,teamAbbrevs,gamesPlayed,shoots,position,ppUnit,vs,avgPowerplayToi,row,...,PPGF_enemy,PPG/PG_enemy,PPO/PG_enemy,PPTOI/PG_enemy,PEN_enemy,PENMin_enemy,PENSeconds/PG_enemy,PENDrawn/GP_enemy,PPGA_enemy,PKTIO/PG_enemy
2133,Zachary Benson,0,BUF,16,L,2,2.0,BOS,106,0,...,18.0,0.75,3.29166,305.458333,110.0,258.0,645.0,4.029789,10.0,368.791666
2134,Dylan Cozens,4,BUF,24,R,3,1.0,BOS,130,1,...,18.0,0.75,3.29166,305.458333,110.0,258.0,645.0,4.029789,10.0,368.791666
2135,Casey Mittelstadt,3,BUF,26,L,3,1.0,BOS,194,2,...,18.0,0.75,3.29166,305.458333,110.0,258.0,645.0,4.029789,10.0,368.791666
2137,JJ Peterka,3,BUF,26,L,2,2.0,BOS,111,4,...,18.0,0.75,3.29166,305.458333,110.0,258.0,645.0,4.029789,10.0,368.791666
2138,Owen Power,2,BUF,26,L,1,1.0,BOS,103,5,...,18.0,0.75,3.29166,305.458333,110.0,258.0,645.0,4.029789,10.0,368.791666


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

features = ['ppPoints', 'gamesPlayed', 'avgPowerplayToi', 'team_PEN/GP', 'team_PP%', 'team_PK%', 'team_GA', 'team_G', 'enemy_PEN/GP', 'enemy_PP%', 'enemy_PK%', 'enemy_GA', 'enemy_G']
label = ['scored']

X_train, X_test, y_train, y_test = train_test_split(df[features], df[label], test_size=0.2, random_state=42, shuffle=True)

# undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)
oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)

# Fit and transform the training data using the RandomUnderSampler
X_train, y_train = oversampler.fit_resample(X_train, y_train)

print(y_train)



      scored
0          0
1          0
2          1
3          0
4          0
...      ...
1427       1
1428       1
1429       1
1430       1
1431       1

[1432 rows x 1 columns]


In [4]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))
acc = accuracy_score(y_test, y_pred)
print(f'Accuracy: {acc}%')

C:\Users\Logan\AppData\Local\Temp\ipykernel_51220\1607553185.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


              precision    recall  f1-score   support

           0       0.94      0.84      0.89       194
           1       0.37      0.62      0.46        29

    accuracy                           0.81       223
   macro avg       0.65      0.73      0.67       223
weighted avg       0.86      0.81      0.83       223

Accuracy: 0.8116591928251121%


## Predict List of Players

In [5]:
df = pd.read_csv('../../../lib/ai_bum_list.csv')
df = df.dropna()
df['avgPowerplayToi'] = df['avgPowerplayToi'].apply(lambda x: datetime.strptime(str(x), "%M:%S").second + datetime.strptime(str(x), "%M:%S").minute * 60)
df['position'].replace(['C', 'R', 'L', 'D'], [3, 2, 2, 1], inplace=True)

In [6]:
features = ['ppPoints', 'gamesPlayed', 'avgPowerplayToi', 'team_PEN/GP', 'team_PP%', 'team_PK%', 'team_GA', 'team_G', 'enemy_PEN/GP', 'enemy_PP%', 'enemy_PK%', 'enemy_GA', 'enemy_G']
label = ['scored']

y_pred = model.predict(df[features])

for i in range(0, len(df)):
    df.loc[i, 'will_score'] = 'Yes' if y_pred[i] == 1 else 'No'

# Players who will score a powerplay point
display = ['skaterFullName', 'ppPoints', 'ppUnit', 'gamesPlayed', 'avgPowerplayToi', 'teamAbbrevs', 'vs', 'will_score']
df[display].sort_values(by=['will_score'], ascending=False)
df.to_csv('scoring_list.csv')

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- PEN
- PENDrawn/GP
- PENDrawn/GP_enemy
- PENMin
- PENMin_enemy
- ...


## Predict Single Player

In [ ]:
player = [{
    'skaterFullName': 'Noah Dobson',
    'ppPoints': 5,
    'teamAbbrevs': 'NYI',
    'gamesPlayed': 13,
    'shoots': 'R',
    'position': 'D',
    'ppUnit': 1,
    'vs': 'EDM',
    'avgPowerplayToi': '3:00',
    'row': 1,
    'team_PEN/GP': 4,
    'team_PP%': 17.1,
    'team_PK%': 76.7,
    'team_G': 32,
    'team_GA': 40,
    'enemy_PEN/GP': 4.77,
    'enemy_PP%': 23.9,
    'enemy_PK%': 70,
    'enemy_G': 35,
    'enemy_GA': 51
}]

df = pd.DataFrame(player)
df = df.dropna()
df['avgPowerplayToi'] = df['avgPowerplayToi'].apply(lambda x: datetime.strptime(str(x), "%M:%S").second + datetime.strptime(str(x), "%M:%S").minute * 60)
df['position'].replace(['C', 'R', 'L', 'D'], [3, 2, 2, 1], inplace=True)

In [ ]:
features = ['ppPoints', 'gamesPlayed', 'avgPowerplayToi', 'team_PEN/GP', 'team_PP%', 'team_PK%', 'team_GA', 'team_G', 'enemy_PEN/GP', 'enemy_PP%', 'enemy_PK%', 'enemy_GA', 'enemy_G']
label = ['scored']

y_pred = model.predict(df[features])
print(f"Prediction: {df['skaterFullName'].values.all()} {'WILL' if y_pred[0] == 1 else 'WILL NOT'} score a powerplay point")